## Local Producer

In this exercise, you'll run a producer on your computer to send some arbitrary files to a topic

In [1]:
# imports
import pathlib, logging, importlib, time, datetime
from threading import Thread
from openmsitoolbox.logging import OpenMSILogger
from openmsistream import UploadDataFile, DataFileUploadDirectory

In [2]:
logger = OpenMSILogger("LocalProducer", filelevel=None)
importlib.reload(logging)

<module 'logging' from '/usr/local/anaconda3/envs/sensorpush/lib/python3.9/logging/__init__.py'>

In [3]:
TOPIC_NAME = "test"
repo_root_dir = pathlib.Path().resolve().parent
CONFIG_FILE_PATH = repo_root_dir/"config_files"/"confluent_cloud_broker.config"
TEST_FILE_DIR = repo_root_dir.parent / "test_files"

In [4]:
for iuf, upload_file_path in enumerate(TEST_FILE_DIR.glob("*")):
    upload_file = UploadDataFile(upload_file_path, logger=logger)
    upload_file.upload_whole_file(CONFIG_FILE_PATH,TOPIC_NAME)

[LocalProducer 2023-12-07 14:43:01] Uploading /Users/margareteminizer/Desktop/dmref_materials_project/openmsistream_short_course/test_files/testing_1.txt to test in 524288-byte chunks using 2 threads....
[LocalProducer 2023-12-07 14:43:01] Waiting for all enqueued messages to be delivered (this may take a moment)....
[LocalProducer 2023-12-07 14:43:01] Done uploading /Users/margareteminizer/Desktop/dmref_materials_project/openmsistream_short_course/test_files/testing_1.txt
[LocalProducer 2023-12-07 14:43:01] Uploading /Users/margareteminizer/Desktop/dmref_materials_project/openmsistream_short_course/test_files/testing_2.txt to test in 524288-byte chunks using 2 threads....
[LocalProducer 2023-12-07 14:43:01] Waiting for all enqueued messages to be delivered (this may take a moment)....
[LocalProducer 2023-12-07 14:43:02] Done uploading /Users/margareteminizer/Desktop/dmref_materials_project/openmsistream_short_course/test_files/testing_2.txt
[LocalProducer 2023-12-07 14:43:02] Uploadin

In [5]:
def upload_task(upload_directory,*args,**kwargs):
    start_time = datetime.datetime.now()
    uploaded_filepaths = upload_directory.upload_files_as_added(*args,**kwargs)
    end_time = datetime.datetime.now()
    ts_format = "%m-%d-%Y %H:%M:%S"
    start_stamp = start_time.strftime(ts_format)
    end_stamp = end_time.strftime(ts_format)
    msg = f"The following files were uploaded between {start_stamp} and {end_stamp}:\n\t"
    msg+="\n\t".join([str(fp) for fp in uploaded_filepaths])
    upload_directory.logger.info(msg)

dfud = DataFileUploadDirectory(TEST_FILE_DIR,CONFIG_FILE_PATH,logger=logger)
upload_thread = Thread(target=upload_task, args=(dfud, TOPIC_NAME,))

In [6]:
upload_thread.start()

[LocalProducer 2023-12-07 14:43:07] Will upload new files added to/Users/margareteminizer/Desktop/dmref_materials_project/openmsistream_short_course/test_files to the test topic as 524288-byte chunks using 2 threads


In [7]:
dfud.control_command_queue.put("q")
upload_thread.join()

[LocalProducer 2023-12-07 14:43:38] Will quit after all currently enqueued files are done being transferred.
[LocalProducer 2023-12-07 14:43:38] Waiting for all enqueued messages to be delivered (this may take a moment)
[LocalProducer 2023-12-07 14:43:38] The following files were uploaded between 12-07-2023 14:43:07 and 12-07-2023 14:43:38:
	testing_2_copy.txt
